## Closing Price

In [ ]:
import torch
import pandas as pd
import joblib
import torch.nn as nn
import numpy as np
from datetime import datetime, timedelta
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from peft import PeftModel

# 1. Define Transformer model
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).float().unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(np.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:x.size(1), :]

class StockGenWithSentimentProbs(nn.Module):
    def __init__(self, input_dim=10, num_layers=4):
        super().__init__()
        self.input_proj = nn.Linear(input_dim, 64)
        self.positional_encoding = PositionalEncoding(d_model=64)
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=64, nhead=4, dim_feedforward=128, dropout=0.1, activation='gelu'),
            num_layers=num_layers
        )
        self.layer_norm = nn.LayerNorm(64)
        self.fc = nn.Linear(64, 1)

    def forward(self, x):
        x = self.input_proj(x)
        x = self.positional_encoding(x)
        x = x.permute(1, 0, 2)
        x = self.transformer(x)
        x = x.permute(1, 0, 2)
        x = self.layer_norm(x)
        x = x.mean(dim=1)
        return self.fc(x).squeeze(-1)

# 2. Load data and model
df = pd.read_csv("sentiment_data_news_ctsh.csv")
# Compute volatility: max(close_t-1 to t-7) - min(close_t-1 to t-7)
def compute_volatility(row):
    try:
        close_prices = [row[f"Close_t-{t}"] for t in range(1, 8)]
        return max(close_prices) - min(close_prices)
    except:
        return 0.0  # Default to 0 if any error

# Apply to dataframe
df["Volatility"] = df.apply(compute_volatility, axis=1)

scaler = joblib.load("stock_global_scaler.pkl")
ticker = df['Ticker'].iloc[0]
df["Close"] = scaler.transform(df[["Close"]])

# 3. Feature engineering
for lag in range(1, 8):
    df[f"Close_t-{lag}"] = df["Close"].shift(lag)
df.dropna(inplace=True)

# 4. Prepare sequence
feature_columns = [f"Close_t-{i}" for i in range(1, 8)] + ["FinBERT_neutral", "FinBERT_positive", "FinBERT_negative"]
sequence = [[row[col] for col in feature_columns] for _, row in df.iterrows()]
X = torch.tensor([sequence], dtype=torch.float32)

# 5. Load transformer model and predict
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = StockGenWithSentimentProbs(input_dim=10, num_layers=4).to(device)
model.load_state_dict(torch.load("stock_model_with_probs.pth", map_location=device))
model.eval()

with torch.no_grad():
    predicted_change = model(X.to(device)).item()

last_close_scaled = df.iloc[-1]["Close_t-1"]
last_close_unscaled = scaler.inverse_transform([[last_close_scaled]])[0][0]
predicted_price = last_close_unscaled + predicted_change

# 6. Determine predicted trend
delta = predicted_price - last_close_unscaled
threshold = 0
if delta > threshold:
    predicted_trend = "Uptrend"
elif delta < -threshold:
    predicted_trend = "Downtrend"
else:
    predicted_trend = "Sideways"

# 7. Determine prediction date
last_date_str = df.iloc[-1]["Date"]
last_date = datetime.strptime(last_date_str, "%Y-%m-%d")
next_date = last_date + timedelta(days=1)

# 8. Use entire CSV + prediction to infer confidence
# Load tokenizer and PEFT model
base_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")
peft_model = PeftModel.from_pretrained(base_model, "./finetuned_lora_model_confidence").to(device)
peft_model.eval()

# Construct a single inference prompt
input_text = (
    f"You are a financial analyst model evaluating prediction reliability.\n"
    f"Ticker: {ticker}\n"
    f"Past Data: {df[feature_columns + ['Close']].tail(10).to_dict(orient='records')}\n"
    f"Predicted Close: {predicted_price:.2f}\n"
    f"Predicted Trend: {predicted_trend}\n"
    f"Last Known Close: {last_close_unscaled:.2f}\n"
    f"Please return a confidence score (0-1) for this prediction."
)

# Tokenize and predict confidence
inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
with torch.no_grad():
    output_ids = peft_model.generate(
        **inputs,
        max_length=16,
        temperature=0.7,    
        top_p=0.9,           
        do_sample=True       
    )   
    confidence_score = tokenizer.decode(output_ids[0], skip_special_tokens=True).strip()
last_volatility = df["Volatility"].iloc[-1]

# 9. Generate recommendation using fine-tuned model
# Load recommendation model
peft_model_recommendation = PeftModel.from_pretrained(base_model, "./finetuned_lora_model_recommendation").to(device)
peft_model_recommendation.eval()

# Construct recommendation prompt
recommendation_input_text = (
    f"You are a financial assistant model. Based on the following data, provide a stock recommendation (Buy, Sell, Hold):\n"
    f"Predicted Trend: {predicted_trend}\n"
    f"Confidence: {confidence_score}\n"
    f"Volatility: {last_volatility:.2f}\n"
    f"Predicted Close: {predicted_price:.2f}\n"
    f"Past 7-Day Closing Prices: {df[feature_columns].tail(7).to_dict(orient='records')}\n"
    f"Recommendation (Buy, Sell, Hold):"
)

# Tokenize and predict recommendation
inputs_recommendation = tokenizer(recommendation_input_text, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
with torch.no_grad():
    output_ids_recommendation = peft_model_recommendation.generate(
        **inputs_recommendation,
        max_length=8,
        temperature=0.7,
        top_p=0.9,
        do_sample=True
    )
    predicted_recommendation = tokenizer.decode(output_ids_recommendation[0], skip_special_tokens=True).strip()

# 10. Save prediction with recommendation
prediction_row = {
    "Date": next_date.strftime("%Y-%m-%d"),
    "Ticker": ticker,
    "Last_Close": last_close_unscaled,
    "Predicted_Close_Next_Day": predicted_price,
    "Predicted_Trend": predicted_trend,
    "Predicted_Confidence": confidence_score,
    "Last_Volatility": last_volatility,
    "Predicted_Recommendation": predicted_recommendation
}
prediction_df = pd.DataFrame([prediction_row])
prediction_df.to_csv("predicted_next_day_price.csv", index=False)

# 11. Output
print(f"\n📈 Predicted Close Price for {next_date.strftime('%Y-%m-%d')}: ${predicted_price:.2f}")
print(f"📊 Predicted Trend: {predicted_trend}")
print(f"🔒 Confidence Score: {confidence_score}")
print(f"Volatility: {last_volatility:.2f}")
print(f"💡 Recommendation: {predicted_recommendation}")
print("📁 Saved prediction with recommendation as 'predicted_next_day_price.csv'")


c:\Users\gagan\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
c:\Users\gagan\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(



📈 Predicted Close Price for 2022-06-02: 74.70
📊 Predicted Trend: Downtrend
🔒 Confidence Score: 100.0
Volatility: 3.90
💡 Recommendation: Sell
📁 Saved prediction with recommendation as 'predicted_next_day_price.csv'
